In [1]:

# UTILITARIES
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
from Bio import SeqIO
# TORCH MODULES FOR METRICS COMPUTATION :
import torch
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
#from torchmetrics.classification import MultilabelF1Score
#from torchmetrics.classification import MultilabelAccuracy



In [9]:
class config:
    train_sequences_path = "/Train/C2H2.fasta"

    
    num_labels = 500
    n_epochs = 5
    batch_size = 128
    lr = 0.001
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from transformers import BertModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert").to(config.device)

def get_bert_embedding(
    sequence : str,
    len_seq_limit : int
):
    sequence_w_spaces = ' '.join(list(sequence))
    encoded_input = tokenizer(
        sequence_w_spaces,
        truncation=True,
        max_length=len_seq_limit,
        padding='max_length',
        return_tensors='pt').to(config.device)
    output = model(**encoded_input)
    output_hidden = output['last_hidden_state'][:,0][0].detach().cpu().numpy()
    assert len(output_hidden)==1024
    return output_hidden

/home/k305-1/miniconda3/envs/cryodrgn1/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/k305-1/miniconda3/envs/cryodrgn1/lib/python3.9/site-packages/torch/_utils.py:771: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
print(config.device)

cuda


In [10]:
for file in glob.glob("/Train/*.fasta"):
    name=os.path.basename(file)
    base_name=name.split(sep='.')[0]
    fasta=os.path.join(base_name+'.fasta')
    class config:
        train_sequences_path = os.path.join("/Train/"+name)


        num_labels = 500
        n_epochs = 5
        batch_size = 128
        lr = 0.001

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
    ids_list = []
    embed_vects_list = []
    t0 = time.time()
    checkpoint = 0
    for item in tqdm(fasta_train):
        ids_list.append(item.id)
        embed_vects_list.append(
            get_bert_embedding(sequence = item.seq, len_seq_limit = 1200))
        checkpoint+=1
        if checkpoint>=100:
            df_res = pd.DataFrame(data={"id" : ids_list, "embed_vect" : embed_vects_list})
            np.save(base_name+'_train_ids.npy',np.array(ids_list))
            np.save(base_name+'_train_embeddings.npy',np.array(embed_vects_list))
            checkpoint=0

    np.save(base_name+'_train_ids.npy',np.array(ids_list))
    np.save(base_name+'_train_embeddings.npy',np.array(embed_vects_list))
    print('Total Elapsed Time:',time.time()-t0)

16027it [32:49,  8.14it/s]

Total Elapsed Time: 1970.0063409805298
